# Explicit Alternative Least Square

------------
In the last section, we have used KNN to created the first restaurant recommendation model. In this section I will be using ALS to solve the exact same probelm; and let see if there is any difference between this two methodologies.

-------------

There are two types of ALS that are commonly used:
 * Explicit ALS
 * Implicit ALS

The only difference them is that in Explicit ALS, we have observation on users rating, which is the case for this dataset. But for the case of Implicit ALS, we have to define users rating based on user behaviors, for example, clickdata, time spent on the webpage, and etc.

In this section, I will be focusing on the **Explicit ALS**.

--------------------------

To use Apache Spark, I spent some time to figure out on to set up Spark environment on my local machine. The tutorial I used can be found in [here](https://medium.com/@GalarnykMichael/install-spark-on-windows-pyspark-4498a5d8d66c)


Now let's start. First, let's import the libraries,

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql import Row
import time
import os
os.chdir("D:\\Yelp data") 

Setup the Spark session and import the data

In [5]:
spark = SparkSession \
        .builder \
        .appName("movie recommendation") \
        .getOrCreate()
sc=spark.sparkContext
sc.setCheckpointDir('C:\\Users\\qwerdf\\Desktop\\Coding\\pyspark')
#Add checkpoint becasue it is HDFS at a local machine; memory can be a contrainst in this case.

joint_table=spark.read.load("yelp_train.csv",format='csv',header=True)
joint_table.show(10)

+---+--------------------+--------------------+--------------------+------------+------------+-----------+----------+--------------------+-------+--------------------+-----------+------------------+--------------+-----------------+-----------+----------+---------+----+
|_c0|           review_id|             user_id|         business_id|stars_review|useful_final|funny_final|cool_final|          categories|   city|                name|postal_code|review_count_final|stars_business|review_count_user|useful_user|funny_user|cool_user|fans|
+---+--------------------+--------------------+--------------------+------------+------------+-----------+----------+--------------------+-------+--------------------+-----------+------------------+--------------+-----------------+-----------+----------+---------+----+
|  6|ymAUG8DZfQcFTBSOi...|u0LXt3Uea_GidxRW1...|9_CGhHMz8698M9-Pk...|           4|           0|          0|         2|Restaurants, Viet...|Toronto|Little Coxwell Vi...|    M4C 3G5|           

First I will need to convert **business_id** and **user_id** into a integer unique identifer, because this is what spark.ALS would take. 

---

Here I leverage the rank over combination in SQL to assign a unique identifier for each user and business.

In [20]:

ALS_data=joint_table.withColumn("business_id_int",rank().over(Window.orderBy(asc("business_id")))) \
                    .withColumn("user_id_int",rank().over(Window.orderBy(asc("user_id")))) \
                    .withColumn("stars_review_int",joint_table["stars_review"].cast(IntegerType())) \
                    .select(["business_id_int","user_id_int","stars_review_int","name"]).cache()

ALS_data.createOrReplaceTempView("ALS_data")
ALS_data.show(10)

+---------------+-----------+----------------+--------------------+
|business_id_int|user_id_int|stars_review_int|                name|
+---------------+-----------+----------------+--------------------+
|         130673|          1|               1|Posticino Ristorante|
|          91180|          2|               4|        The Pie Guyz|
|         345830|          3|               2| Slumdog Bar & Grill|
|         137697|          4|               2|                Cava|
|         128641|          5|               3|      Sansotei Ramen|
|         132919|          5|               4|CoCo Fresh Tea & ...|
|         178897|          5|               5|    Wooffles & Cream|
|         438575|          5|               2|          Little Fin|
|           5406|          9|               1|The Works Gourmet...|
|          25789|          9|               5|          Cafe Jules|
+---------------+-----------+----------------+--------------------+
only showing top 10 rows



Ensure that every line has **business_id**, **review score** and **user id**

In [8]:
train,validation=ALS_data.randomSplit([0.8,0.2],seed=1992)
train.createOrReplaceTempView("train")
spark.sql(
    "Select * \
     FROM  train \
     WHERE stars_review_int is null or business_id_int is null or user_id_int is null"
).show(10)

+-----------+---------------+-------+-----------+----------------+----+
|business_id|business_id_int|user_id|user_id_int|stars_review_int|name|
+-----------+---------------+-------+-----------+----------------+----+
+-----------+---------------+-------+-----------+----------------+----+



Define the function for ALS parameter tuning. The two parameter needs to be tuned are $ lambda $, which is the regularization parameter that prevent the model from overfitting; and the $ rank $, which defines the complexity of the model.

Root mean square error is the error function used for ALS.

In [5]:
def ALS_parameter_tuning(train_data,validation_data,maxiter,reg_param,rank):
        min_error=float('inf')
        best_rank=-1
        best_regularization=0,
        best_model=None
        run_time=time.time()
        for rank_iter in ranks:
            for reg in reg_param:
                start_time=time.time()
                als=ALS(
                        maxIter=maxiter,
                        regParam=reg,
                        userCol="user_id_int",
                        itemCol="business_id_int",
                        ratingCol="stars_review_int",
                        rank=rank_iter,
                        coldStartStrategy="drop",
                        checkpointInterval=2
                        )
                model=als.fit(train_data)
                predictions=model.transform(validation_data)
                #use rmse to calculate the error term for each model; The smaller the better.
                evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars_review_int",
                                              predictionCol="prediction")
                rmse=evaluator.evaluate(predictions)
                print("With reg Param={} and rank={}, RMSE of ALS is {}".format(reg,rank_iter,rmse))
        if rmse<=min_error:
            min_error=rmse
            best_rank=rank_iter
            best_regularization=reg
            best_model=model
        run_time=time.time()-run_time
        return best_rank,best_regularization,best_model,run_time

In [6]:
maxiter=10
ranks=range(10,21,2)
start_time = time.time()
reg_params=[0.02,0.05,0.1,0.2,0.3,0.5]

In [7]:
best_rank,best_regularization,best_model,run_time=ALS_parameter_tuning(train_data=train,validation_data=validation,maxiter=maxiter,reg_param=reg_params,rank=ranks)

With reg Param=0.02 and rank=10, RMSE of ALS is 1.7949761352739237
With reg Param=0.05 and rank=10, RMSE of ALS is 1.5408083354043882
With reg Param=0.1 and rank=10, RMSE of ALS is 1.3971850282005798
With reg Param=0.2 and rank=10, RMSE of ALS is 1.2986664217191408
With reg Param=0.3 and rank=10, RMSE of ALS is 1.2706742549255283
With reg Param=0.5 and rank=10, RMSE of ALS is 1.2871139932253073
With reg Param=0.02 and rank=12, RMSE of ALS is 1.8167712994440626
With reg Param=0.05 and rank=12, RMSE of ALS is 1.5469455633972993
With reg Param=0.1 and rank=12, RMSE of ALS is 1.4013181306592983
With reg Param=0.2 and rank=12, RMSE of ALS is 1.3018688871840587
With reg Param=0.3 and rank=12, RMSE of ALS is 1.2725680032318314
With reg Param=0.5 and rank=12, RMSE of ALS is 1.2878969365715573
With reg Param=0.02 and rank=14, RMSE of ALS is 1.7851608968941934
With reg Param=0.05 and rank=14, RMSE of ALS is 1.5342439796362384
With reg Param=0.1 and rank=14, RMSE of ALS is 1.3946430205670426
With

From the result, we can see that when $ Lambda $ =0.3 and rank=20, ALS gives the best RMSE at 1.27

What we can observe is that RMSE is montonic with regard to rank. The higher the rank the lower the RMSE; that makes perfect as the more complex the model is, the more accurate it would be. However, we observe there is a local minimum for RMSE when **lambda** is changed. This is becasue **lambda** is the regularization factor for overfitting the model. The higher the **lambda** is, the more penalty it would put on the model for overfitting. In this case, the model performs worse when **lambda = 0.5** than when **lambda=0.3** becasue the model lost too much detail in describing the data to prevent it from overfitting.

----------------------
Now, we have the best model, let's use ALS to make some recommendations. First, I will need a map to convert restaurant name into the integer business id

In [17]:
def find_restaurant_id(data,restaurant_list):
    """
    
    
    """
    final_list=[]
    if restaurant_list is None:
        return final_list
    for restaurant in restaurant_list:
        restaurant_id=data.where(col('name') \
                          .like(restaurant)) \
                          .select('business_id_int') \
                          .distinct() \
                          .rdd \
                          .map(lambda r:r[0]) \
                          .collect()    
                          
        final_list.extend(restaurant_id)
    return final_list

Next, define the function that would add new data into the model.

In [10]:
def add_new_user_data(train_data,my_fav,my_least_fav=None):
    """
    function to add new user's data into train model
    
    
    --------------
    
    
    parameters:
    
    train_data: spark dataframe, Initial data for user_id,business_id,rating combinations
    
    my_fav: list of strings, favourite restaurant list; Assume these restaurants are marked as 5
    
    my_least_fav: list of strings, least favourite restaurant list; Assume these restaurants are makred as 1
    """
    new_user_id=train_data.rdd.map(lambda r:r[1]).max()+1
    columns=['business_id_int','user_id_int','stars_review','name']
    print("initialize data for new userid {}".format(new_user_id))
    fav_list=find_restaurant_id(train_data,my_fav)
    fav_rows=[(restaurant_id,new_user_id,5,restaurant) for restaurant_id,restaurant in zip(fav_list,my_fav)]
    fav_df=spark.createDataFrame(fav_rows,columns)
    final_set=train_data.union(fav_df)
    if my_least_fav is not None:
        least_fav_list=find_restaurant_id(train_data,my_least_fav)
        least_fav_rows=[(restaurant_id,new_user_id,1,restaurant) for restaurant_id,restaurant in zip(least_fav_list,my_least_fav)]
        least_fav_df=spark.createDataFrame(least_fav_rows,columns)
        final_set=train_data.union(least_fav_df)
    return new_user_id,final_set

In [22]:
def find_personal_recommendation_als(train_data,my_fav,my_least_fav=None,reg_param=0.3,rank=20,maxiter=10):
    """
    function to give personal recommendations to the defined user who has record of favourite restaurant and least favourite restaurant
    
    -----------
    parameters:
    
    train_data: spark dataframe, Initial data for user_id,business_id,rating combinations
    
    my_fav: list of strings, favourite restaurant list; Assume these restaurants are marked as 5
    
    my_least_fav: list of strings, least favourite restaurant list; Assume these restaurants are makred as 1
    
    reg_param: lambda in ALS, defined the overfitting penalty of the ALS
    
    rank: rank of ALS, defined the complexity of ALS
    
    
    """
    best_regularization=reg_param
    best_rank=rank
    new_user_id,new_set=add_new_user_data(train_data,my_fav,my_least_fav)
    
    best_als=ALS(maxIter=maxiter, \
             regParam=best_regularization, \
             userCol="user_id_int", \
             itemCol="business_id_int", \
             ratingCol="stars_review_int", \
             rank=best_rank, \
             coldStartStrategy="drop", \
             checkpointInterval=2
             )
    model=best_als.fit(new_set)
    users=new_set.select(best_als.getUserCol()).where(col('user_id_int')==new_user_id)
    userSubsetRecs=model.recommendForUserSubset(users,10)
    print("Here are the top 10 recommandations for you given your favourite and least favourite food")
    for restaurant in userSubsetRecs.select("recommendations").collect()[0][0]:
        restaurant_id=restaurant.__getattr__("business_id_int")
        restaurant_name=train_data.where(col('business_id_int')==restaurant_id).select('name').distinct().collect()
        print(restaurant_name[0].__getattr__("name"))
    return userSubsetRecs


In [23]:
my_fav=['Emerald Chinese Restaurant']

find_personal_recommendation_als(ALS_data,my_fav)

initialize data for new userid 458670
Here are the top 10 recommandations for you given your favourite and least favourite food
Sushi Making For the Soul
Shawarma Q
Souppe Shoppe
Sakura Ichiban Sushi Express
Ten Ten Dim Sum
Charlotte's Homemade Goodies
Brando's Fried Chicken
Lox + Schmear
Pita Pit
Touch Down Foods


DataFrame[user_id_int: int, recommendations: array<struct<business_id_int:int,rating:float>>]

Compare this to the result from KNN

-------------
(KNN recommendation)

Recommendations for Emerald Chinese Restaurant:

1: Kobe Sushi, with distance of 0.8729435047925027

2: Wonton Chai Noodle, with distance of 0.8717288246806103

3: Chako, with distance of 0.8693951111786435

4: Congee Dynasty, with distance of 0.8658084838350306

5: Dynasty BBQ, with distance of 0.861539455352046

6: McDonald's, with distance of 0.8548495321630935

7: Wild Wing, with distance of 0.854550339648705

8: Blue Lagoon Seafood Master, with distance of 0.8544684454549096

9: Viet Thai Restaurant, with distance of 0.8378466819037798

10: Happy Jade Seafood Chinese Restaurant, with distance of 0.7963704755081871

-------------

It is surprise to see that ALS gives totally different recommendations comparing to KNN. 

Let's take another example of J's Fish and Chips


In [28]:
my_fav=["Ann's Congee"]
find_personal_recommendation_als(ALS_data,my_fav,my_least_fav) 

initialize data for new userid 458670
Here are the top 10 recommandations for you given your favourite and least favourite food
Sushi Making For the Soul
Souppe Shoppe
Shawarma Q
Ten Ten Dim Sum
Charlotte's Homemade Goodies
Sakura Ichiban Sushi Express
Brando's Fried Chicken
Lox + Schmear
Pita Pit
Cuisine of India


DataFrame[user_id_int: int, recommendations: array<struct<business_id_int:int,rating:float>>]

Compared it to the KNN model

-------------------------------------------------------
1: Taza Xpress, with distance of 0.9176311421402781

2: Quesada Burritos and Tacos, with distance of 0.9038392772773323

3: Twisted Fork, with distance of 0.896251268968906

4: Yamato Sushi, with distance of 0.8955883310348774

5: St. Louis Bar & Grill, with distance of 0.8916467512822909

6: Otaru Sushi, with distance of 0.8879336370638948

7: Bakery Garden, with distance of 0.8687116746261058

8: Sunset Grill, with distance of 0.8267602678311907

9: Mrs. Greek Souvlaki Express, with distance of 0.7920633893787208

10: The Prince Albert Pub, with distance of 0.7846347538730261

Again, very different.